In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import warnings
from shapely.geometry import Point, LineString
warnings.filterwarnings('ignore')

In [ ]:
main = r'F:\lampang_2'
gdf_road = gpd.read_file(rf'{main}\shapefile\Road_Lampang.shp')
# gdf_road.drop(columns= ['lane'], inplace = True)
gdf_grid = gpd.read_file(rf'{main}\shapefile\Grid_Lampang.shp')
gdf_grid.to_crs('EPSG:4326', inplace = True)
gdf_grid.drop(columns = 'Id', inplace=True)
grid_la = gpd.read_file(rf'{main}\shapefile\Grid_Lampang_label.shp')
grid_la.to_crs('EPSG:4326', inplace = True)
grid_la['Latitude'] = grid_la.geometry.y
grid_la['Longigude'] = grid_la.geometry.x
grid_la.drop(columns=['geometry', 'Id'], inplace=True)

In [ ]:
road_grid = pd.DataFrame()
for id in range(len(gdf_road.index)):
    print(id)
    gdf_test = gdf_road.iloc[[id]]
    match = gpd.sjoin(gdf_test, gdf_grid)
    match.drop(columns = 'geometry', inplace=True)
    match = pd.merge(match, grid_la, left_on = 'index_right', right_index=True, how = 'left')
    match.rename(columns = {'id' : 'ID_road', 'index_right' : 'ID_Grid'}, inplace=True)
    road_grid = pd.concat([match, road_grid], axis=0)
road_grid.sort_index(inplace = True)

In [ ]:
#Extracting start and end points
gdf = gdf_road
gdf['first'] = None
gdf['last'] = None

for index, row in gdf.iterrows():
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    gdf.at[index,'first'] = Point(first_coord)
    gdf.at[index,'last'] = Point(last_coord)
    gdf
gdf_start = gdf.drop(columns = ['geometry', 'last'], axis = 1)
gdf_start = gdf_start.set_geometry("first")
gdf_last = gdf.drop(columns = ['geometry', 'first'], axis = 1)
gdf_last = gdf_last.set_geometry("last")
ESRI_WKT = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]'
# gdf_start.to_file(filename = rf'{main}\shapefile\start_point.shp', driver = 'ESRI Shapefile',crs = ESRI_WKT)
# gdf_last.to_file(filename = rf'{main}\shapefile\last_point.shp', driver = 'ESRI Shapefile',crs = ESRI_WKT)
gdf.drop(columns = ['geometry'] , axis=1, inplace=True)
gdf = gdf.set_geometry("first")
gdf['start_Latitude'] = gdf.geometry.y
gdf['start_Longigude'] = gdf.geometry.x
gdf = gdf.set_geometry("last")
gdf['end_Latitude'] = gdf.geometry.y
gdf['end_Longigude'] = gdf.geometry.x
gdf['start'] = gdf['start_Latitude'].astype(str) + ',' + gdf['start_Longigude'].astype(str)
gdf['end'] = gdf['end_Latitude'].astype(str) + ',' + gdf['end_Longigude'].astype(str)

In [4]:
writer = pd.ExcelWriter(r'F:\lampang_2\piggy\Example_Import-RoadData_Note.xlsx', engine = 'xlsxwriter')
gdf.to_excel(writer, sheet_name = 'Road')
road_grid.to_excel(writer, sheet_name = 'Grid_Road')

writer.save()
writer.close()

NameError: name 'gdf' is not defined